## **Exemples : Agents Multi-IA pour la réservation d'hôtel**

Dans le monde rapide d'aujourd'hui, planifier un voyage d'affaires implique bien plus que de réserver un vol et une chambre d'hôtel. Cela nécessite un niveau de coordination et d'efficacité qui peut être difficile à atteindre. C'est là que les Agents Multi-IA interviennent, révolutionnant la façon dont nous gérons nos besoins de voyage.

Imaginez avoir une équipe d'agents intelligents à votre disposition, travaillant ensemble pour gérer chaque aspect de votre voyage avec précision et facilité. Grâce à notre technologie avancée d'IA, nous avons créé des agents spécialisés pour les services de réservation et l'organisation d'itinéraire, garantissant une expérience de voyage fluide et sans stress.

Ceci est un scénario de base. Lors de la planification d'un voyage d'affaires, nous devons consulter un agent de voyage d'affaires pour obtenir des informations sur les billets d'avion, les hôtels, etc. Grâce aux Agents IA, nous pouvons créer des agents pour les services de réservation et des agents pour l'organisation d'itinéraire afin de collaborer et d'améliorer le niveau d'intelligence.


# Initialiser le service Azure AI Agent et obtenir les informations de configuration depuis **.env**

### **.env** 

Créer un fichier .env

**.env** contient la chaîne de connexion du service Azure AI Agent, le modèle utilisé par AOAI, et le service API de recherche Google correspondant, ENDPOINT, etc.

- **AZURE_AI_AGENT_MODEL_DEPLOYMENT_NAME** = "Nom de déploiement du modèle du service Azure AI Agent"

[**NOTE**] Vous aurez besoin d’un modèle avec une limite de taux de 100 000 (jetons par minute) et une limite de taux de 600 (requêtes par minute)

  Vous pouvez obtenir un modèle dans Microsoft Foundry - Model and Endpoint.

- **AZURE_AI_AGENT_PROJECT_CONNECTION_STRING** = "Chaîne de connexion du projet du service Azure AI Agent"

  Vous pouvez obtenir la chaîne de connexion du projet dans la vue d’ensemble de votre projet dans l’écran du portail AI Foundry.

- **SERPAPI_SEARCH_API_KEY** = "Votre clé API SERPAPI Search"
- **SERPAPI_SEARCH_ENDPOINT** = "Votre endpoint SERPAPI Search"

Pour obtenir le nom de déploiement du modèle et la chaîne de connexion du projet du service Azure AI Agent, vous devez créer le service Azure AI Agent. Il est recommandé d’utiliser [ce modèle](https://portal.azure.com/#create/Microsoft.Template/uri/https%3A%2F%2Fraw.githubusercontent.com%2Ffosteramanda%2Fazure-agent-quickstart-templates%2Frefs%2Fheads%2Fmaster%2Fquickstarts%2Fmicrosoft.azure-ai-agent-service%2Fstandard-agent%2Fazuredeploy.json) pour le créer directement （***Note :*** Le service Azure AI Agent est actuellement configuré dans une région limitée. Il est recommandé de consulter [ce lien](https://learn.microsoft.com/en-us/azure/ai-services/agents/concepts/model-region-support) pour définir la région)

L’agent doit accéder à SERPAPI. Il est recommandé de s’inscrire via [ce lien](https://serpapi.com/searches). Après inscription, vous pouvez obtenir une clé API unique et un ENDPOINT.


# Connexion à Azure 

Vous devez maintenant vous connecter à Azure. Ouvrez un terminal dans VScode et exécutez la commande `az login`


# Configuration

Pour exécuter ce carnet, vous devrez installer les bibliothèques suivantes. Voici une liste des bibliothèques requises et des commandes pip install correspondantes :

azure-identity : Pour l'authentification Azure.  
requests : Pour effectuer des requêtes HTTP.  
semantic-kernel : Pour le framework de noyau sémantique (en supposant qu'il s'agit d'une bibliothèque personnalisée ou spécifique, vous pourriez devoir l'installer à partir d'une source ou d'un dépôt spécifique).


In [ ]:
!pip install azure-identity
!pip install requests
!pip install semantic-kernel
!pip install --upgrade semantic_kernel
!pip install azure-cli

# Explication : 
import asyncio : Cela importe le module asyncio, qui offre un support pour la programmation asynchrone en Python. Il vous permet d’écrire du code concurrent en utilisant la syntaxe async et await.  
from typing import Annotated : Cela importe le type Annotated depuis le module typing. Annotated est utilisé pour ajouter des métadonnées aux annotations de type, ce qui peut être utile pour diverses finalités comme la validation, la documentation ou les outils.


In [ ]:
import asyncio,os
from typing import Annotated

# Explication :
En utilisant from dotenv import load_dotenv et load_dotenv(), vous pouvez facilement gérer les paramètres de configuration et les informations sensibles (comme les clés API et les URL de base de données) dans un fichier .env, les gardant séparés de votre code source et rendant votre application plus sécurisée et plus facile à configurer.


In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Explication :

Instruction d'importation : from azure.identity.aio import DefaultAzureCredential : Cela importe la classe DefaultAzureCredential depuis le module azure.identity.aio. La partie aio du nom du module indique qu'il est conçu pour les opérations asynchrones.

Objet de DefaultAzureCredential : La classe DefaultAzureCredential fait partie du SDK Azure pour Python. Elle offre un moyen par défaut de s'authentifier auprès des services Azure. Elle tente de s'authentifier en utilisant plusieurs méthodes dans un ordre spécifique, telles que les variables d'environnement, l'identité managée et les identifiants Azure CLI.

Opérations asynchrones : Le module aio indique que la classe DefaultAzureCredential prend en charge les opérations asynchrones. Cela signifie que vous pouvez l'utiliser avec asyncio pour effectuer des requêtes d'authentification non bloquantes.


In [ ]:
from azure.identity.aio import DefaultAzureCredential

# Explication :
Importe divers modules et classes du package semantic_kernel. Voici une répartition de chaque importation :

AgentGroupChat de semantic_kernel.agents : Cette classe gère les fonctionnalités liées au chat de groupe pour les agents IA. AzureAIAgent et AzureAIAgentSettings de semantic_kernel.agents.azure_ai

AzureAIAgent : Cette classe est utilisée pour créer et gérer des agents IA qui utilisent les services Azure AI.

AzureAIAgentSettings : Cette classe est utilisée pour configurer les paramètres pour AzureAIAgent. TerminationStrategy de semantic_kernel.agents.strategies.termination.termination_strategy :

Cette classe définit des stratégies pour terminer l'exécution des agents IA sous certaines conditions. ChatMessageContent de semantic_kernel.contents.chat_message_content :

Cette classe est utilisée pour gérer le contenu des messages de chat.
AuthorRole de semantic_kernel.contents.utils.author_role :

Cette classe définit différents rôles pour les auteurs dans le contexte des messages de chat.

kernel_function de semantic_kernel.functions.kernel_function_decorator : Ce décorateur est utilisé pour définir des fonctions kernel, qui sont des fonctions pouvant être exécutées dans le cadre du framework semantic kernel.
Ces importations configurent les composants nécessaires pour créer et gérer des agents IA capables d'interagir dans un environnement de chat de groupe, éventuellement pour des tâches telles que la réservation d'hôtels ou des activités similaires.


In [ ]:
from semantic_kernel.agents import AgentGroupChat
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentSettings
from semantic_kernel.agents.strategies.termination.termination_strategy import TerminationStrategy
from semantic_kernel.contents import ChatMessageContent
from semantic_kernel.contents import AuthorRole
from semantic_kernel.functions.kernel_function_decorator import kernel_function

# Explication :
Ensuite, nous importons la classe CodeInterpreterTool du module azure.ai.projects.models.

CodeInterpreterTool : Cette classe fait partie du SDK Azure AI et est utilisée pour interpréter et exécuter du code dans le contexte des projets d'IA. Elle fournit des fonctionnalités pour exécuter des extraits de code, analyser du code ou intégrer l'exécution de code dans les flux de travail d'IA.
Cet import configure le composant nécessaire pour utiliser CodeInterpreterTool dans votre projet, ce qui peut être utile pour les tâches impliquant l'interprétation et l'exécution dynamique de code.


In [ ]:
from azure.ai.projects.models import CodeInterpreterTool

# Explication :  
La classe ApprovalTerminationStrategy fournit une stratégie spécifique pour terminer l'opération d'un agent IA. L'agent s'arrêtera si le dernier message dans son historique d'interaction contient le mot "saved". Cela peut être utile dans des scénarios où la tâche de l'agent est considérée comme terminée une fois qu'il reçoit la confirmation que quelque chose a été "saved".Définir la méthode d'interaction. Après que le plan de réservation est sauvegardé, il peut être arrêté à la réception du signal saved.


In [ ]:
class ApprovalTerminationStrategy(TerminationStrategy):
    """A strategy for determining when an agent should terminate."""

    async def should_agent_terminate(self, agent, history):
        """Check if the agent should terminate."""
        return "saved" in history[-1].content.lower()

# Explication :

La ligne de code initialise un objet AzureAIAgentSettings avec des paramètres par défaut ou prédéfinis en appelant la méthode create(). Cet objet de paramètres (ai_agent_settings) peut ensuite être utilisé pour configurer et gérer une instance AzureAIAgent.


In [ ]:
ai_agent_settings = AzureAIAgentSettings.create()

# Explication :
En important la bibliothèque requests, vous pouvez facilement effectuer des requêtes HTTP et interagir avec des services web dans votre code Python.


In [ ]:
import requests

# Explication :
Ceci est une variable qui stocke la clé API pour accéder à un service API SERP (page de résultats d’un moteur de recherche). Une clé API est un identifiant unique utilisé pour authentifier les requêtes associées à votre compte.

'GOOGLE_SEARCH_API_KEY' : Ceci est une chaîne de caractères fictive. Vous devez remplacer ''GOOGLE_SEARCH_API_KEY' par votre clé API SERP réelle.

But : Le but de cette ligne est de stocker la clé API dans une variable afin qu’elle puisse être utilisée pour authentifier les requêtes vers le service API SERP. La clé API est requise pour accéder au service et effectuer des recherches.

Comment obtenir une clé API SERP : Pour obtenir une clé API SERP, suivez ces étapes générales sur https://serpapi.com (les étapes exactes peuvent varier selon le service API SERP spécifique que vous utilisez) :

Choisir un service API SERP : Plusieurs services API SERP sont disponibles, tels que SerpAPI, Google Custom Search JSON API, et d’autres. Choisissez celui qui correspond le mieux à vos besoins.

S’inscrire pour un compte :

Rendez-vous sur le site web du service API SERP choisi https://www.serpapi.com et inscrivez-vous pour un compte. Il se peut que vous deviez fournir quelques informations de base et vérifier votre adresse e-mail.

Créer une clé API :

Après l’inscription, connectez-vous à votre compte et naviguez vers la section API ou le tableau de bord. Cherchez une option pour créer ou générer une nouvelle clé API.
Copier la clé API :

Une fois la clé API générée, copiez-la. Cette clé sera utilisée pour authentifier vos requêtes vers le service API SERP.
Remplacer le texte de substitution :

Remplacez le texte de substitution dans votre fichier .env


In [ ]:
SERPAPI_SEARCH_API_KEY=os.getenv('SERPAPI_SEARCH_API_KEY')

In [ ]:
SERPAPI_SEARCH_ENDPOINT = os.getenv('SERPAPI_SEARCH_ENDPOINT')

# Explication :
La classe BookingPlugin fournit des méthodes pour réserver des hôtels et des vols en utilisant l'API de recherche Google Serpapi.com. Elle construit les paramètres nécessaires, envoie des requêtes à l'API et traite les réponses pour retourner des informations pertinentes sur les réservations. La clé API (SERPAPI_SEARCH_API_KEY) et le point de terminaison (SERPAPI_SEARCH_ENDPOINT) sont utilisés pour authentifier et envoyer les requêtes à l'API de recherche Google.


In [ ]:
# Define Booking Plugin
class BookingPlugin:
    """Booking Plugin for customers"""
    @kernel_function(description="booking hotel")
    def booking_hotel(self,query: Annotated[str, "The name of the city"], check_in_date: Annotated[str, "Hotel Check-in Time"], check_out_date: Annotated[str, "Hotel Check-in Time"])-> Annotated[str, "Return the result of booking hotel infomation"]:

        params = {
            "engine": "google_hotels",
            "q": query,
            "check_in_date": check_in_date,
            "check_out_date": check_out_date,
            "adults": "2",
            "currency": "USD",
            "gl": "us",
            "hl": "en",
            "api_key": SERPAPI_SEARCH_API_KEY
        }

        response = requests.get(SERPAPI_SEARCH_ENDPOINT, params=params)
        if response.status_code == 200:
            response = response.json()
            return response["properties"]
        else:
            return None

    
    @kernel_function(description="booking fight")
    def  booking_fight(self,origin: Annotated[str, "The name of Departure"], destination: Annotated[str, "The name of Destination"], outbound_date: Annotated[str, "The date of outbound"], return_date: Annotated[str, "The date of Return_date"])-> Annotated[str, "Return the result of booking fight infomation"]:
        
        go_params = {
            "engine": "google_flights",   
            "departure_id": origin,
            "arrival_id": destination,
            "outbound_date": outbound_date,
            "return_date": return_date,  
            "currency": "USD",
            "hl": "en",
            "api_key": SERPAPI_SEARCH_API_KEY  
        }

        print(go_params)

        go_response = requests.get(SERPAPI_SEARCH_ENDPOINT, params=go_params)


        result = ''

        if go_response.status_code == 200:
            response = go_response.json()

            result += "# outbound \n " + str(response)
        else:
            print('error!!!')
            # return None

        
        back_params = {
            "engine": "google_flights",   
            "departure_id": destination,
            "arrival_id": origin,
            "outbound_date": return_date,
            "return_date": return_date,  
            "currency": "USD",
            "hl": "en",
            "api_key": SERPAPI_SEARCH_API_KEY  
        }


        print(back_params)


        back_response = requests.get(SERPAPI_SEARCH_ENDPOINT, params=back_params)



        if back_response.status_code == 200:
            response = back_response.json()

            result += "\n # return \n"  + str(response)

        else:
            print('error!!!')
            # return None
        
        print(result)

        return result

        


# Explication :
La classe SavePlugin fournit une méthode saving_plan pour enregistrer des plans de voyage en utilisant les services Azure AI. Elle configure les identifiants Azure, crée un agent IA, traite les entrées utilisateur pour générer et enregistrer le contenu du plan de voyage, et gère les opérations d'enregistrement des fichiers et de nettoyage. La méthode retourne "Saved" en cas de succès.


In [ ]:
class SavePlugin:
    """Save Plugin for customers"""
    @kernel_function(description="saving plan")
    async def saving_plan(self,tripplan: Annotated[str, "The content of trip plan"])-> Annotated[str, "Return status of save content"]:

        async with (
            DefaultAzureCredential() as creds,
            AzureAIAgent.create_client(
                credential=creds,
                conn_str=ai_agent_settings.project_connection_string.get_secret_value(),
            ) as client,
        ):

            code_interpreter = CodeInterpreterTool()
            
            agent_definition = await client.agents.create_agent(
                model=ai_agent_settings.model_deployment_name,
                tools=code_interpreter.definitions,
                tool_resources=code_interpreter.resources,
            )


            agent = AzureAIAgent(
                client=client,
                definition=agent_definition,
            )

            thread = await client.agents.create_thread()


            user_inputs = [
                """
            
                        You are my Python programming assistant. Generate code,save """+ tripplan +
                        
                    """    
                        and execute it according to the following requirements

                        1. Save blog content to trip-{YYMMDDHHMMSS}.md

                        2. give me the download this file link
                    """
            ]



            try:
                for user_input in user_inputs:
                    # Add the user input as a chat message
                    await agent.add_chat_message(
                        thread_id=thread.id, message=ChatMessageContent(role=AuthorRole.USER, content=user_input)
                    )
                    print(f"# User: '{user_input}'")
                    # Invoke the agent for the specified thread
                    async for content in agent.invoke(thread_id=thread.id):
                        if content.role != AuthorRole.TOOL:
                            print(f"# Agent: {content.content}")

                    
                    messages = await client.agents.list_messages(thread_id=thread.id)

                    # OpenAIPageableListOfThreadMessage
                    # OpenAIPageableListOfThreadMessage


                    for file_path_annotation in messages.file_path_annotations:

                            file_name = os.path.basename(file_path_annotation.text)

                            await client.agents.save_file(file_id=file_path_annotation.file_path.file_id, file_name=file_name,target_dir="./trip")

                    
            finally:
                await client.agents.delete_thread(thread.id)
                await client.agents.delete_agent(agent.id)


        return "Saved"

# Explication :
Ce code configure des agents Azure AI pour gérer la réservation de vols et d'hôtels, ainsi que la sauvegarde des plans de voyage en fonction des entrées utilisateur. Il utilise les identifiants Azure pour créer et configurer les agents, traite les entrées utilisateur via un chat de groupe, et assure un nettoyage approprié après la réalisation des tâches. Les agents utilisent des plugins spécifiques (BookingPlugin et SavePlugin) pour effectuer leurs tâches respectives.


In [ ]:
async with (
    DefaultAzureCredential() as creds,
    AzureAIAgent.create_client(
        credential=creds,
        conn_str=ai_agent_settings.project_connection_string.get_secret_value(),
    ) as client,
):
    BOOKING_AGENT_NAME = "BookingAgent"
    BOOKING_AGENT_INSTRUCTIONS = """
    You are a booking agent. Help me book flights or hotels.

    Thought: Please understand the user's intention and confirm whether to use the reservation system to complete the task.

    Actions:
    - For flight bookings, convert the departure and destination names into airport codes.
    - Use the appropriate API for hotel or flight bookings. Verify that all necessary parameters are available. If any parameters are missing, ask the user to provide them. If all parameters are complete, call the corresponding function.
    - If the task is not related to hotel or flight booking, respond with the final answer only.
    - Output the results using a markdown table:
      - For flight bookings, output separate outbound and return contents in the order of:
        Departure Airport | Airline | Flight Number | Departure Time | Arrival Airport | Arrival Time | Duration | Airplane | Travel Class | Price (USD) | Legroom | Extensions | Carbon Emissions (kg).
      - For hotel bookings, output in the order of:
        Property Name | Property Description | Check-in Time | Check-out Time | Prices | Nearby Places | Hotel Class | GPS Coordinates.
    """

    SAVE_AGENT_NAME = "SaveAgent"
    SAVE_AGENT_INSTRUCTIONS = """
    You are a save tool agent. Help me to save the trip plan.
    """

    # Create agent definition
    booking_agent_definition = await client.agents.create_agent(
        model=ai_agent_settings.model_deployment_name,
        name=BOOKING_AGENT_NAME,
        instructions=BOOKING_AGENT_INSTRUCTIONS,
    )

    # Create the AzureAI Agent
    booking_agent = AzureAIAgent(
        client=client,
        definition=booking_agent_definition,
        # Optionally configure polling options
        # polling_options=RunPollingOptions(run_polling_interval=timedelta(seconds=1)),
    )

    # Add the sample plugin to the kernel
    booking_agent.kernel.add_plugin(BookingPlugin(), plugin_name="booking")

    # Create agent definition
    save_agent_definition = await client.agents.create_agent(
        model=ai_agent_settings.model_deployment_name,
        name=SAVE_AGENT_NAME,
        instructions=SAVE_AGENT_INSTRUCTIONS
    )

    # Create the AzureAI Agent
    save_agent = AzureAIAgent(
        client=client,
        definition=save_agent_definition,
    )

    save_agent.kernel.add_plugin(SavePlugin(), plugin_name="saving")

    user_inputs = [
        "I have a business trip from London to New York in Feb 20 2025 to Feb 27 2025 ,help me to book a hotel and fight tickets and save it"
    ]

    chat = AgentGroupChat(
        agents=[booking_agent, save_agent],
        termination_strategy=ApprovalTerminationStrategy(agents=[save_agent], maximum_iterations=10),
    )

    try:
        for user_input in user_inputs:
            # Add the user input as a chat message
            await chat.add_chat_message(
                ChatMessageContent(role=AuthorRole.USER, content=user_input)
            )
            print(f"# User: '{user_input}'")

            async for content in chat.invoke():
                print(f"# {content.role} - {content.name or '*'}: '{content.content}'")

            print(f"# IS COMPLETE: {chat.is_complete}")

            print("*" * 60)
            print("Chat History (In Descending Order):\n")
            async for message in chat.get_chat_messages(agent=save_agent):
                print(f"# {message.role} - {message.name or '*'}: '{message.content}'")
    finally:
        await chat.reset()
        await client.agents.delete_agent(save_agent.id)
        await client.agents.delete_agent(booking_agent.id)


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Avertissement** :  
Ce document a été traduit à l’aide du service de traduction automatique [Co-op Translator](https://github.com/Azure/co-op-translator). Bien que nous nous efforcions d’assurer l’exactitude, veuillez noter que les traductions automatiques peuvent contenir des erreurs ou des inexactitudes. Le document original dans sa langue d’origine doit être considéré comme la source faisant autorité. Pour les informations critiques, il est recommandé de faire appel à une traduction professionnelle réalisée par un humain. Nous déclinons toute responsabilité en cas de malentendus ou d’interprétations erronées résultant de l’utilisation de cette traduction.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
